### Adding Realistic Substructure to the Lens Mass Model ###
This notebook adds more realistic perturbers in the form of sub-halos to the mass model of the lens.
First we will import all of the relevant packages and scripts.

In [ ]:
#setting os path to import scripts
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

#importing packages
from astropy.visualization import simple_norm
import corner
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
import numpy as np
from paltas.Configs.config_handler import ConfigHandler
import pickle
from scipy.stats import multivariate_normal
from Scripts import lens_parameters, paltas_model, metrics, network_predictions

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### 1) As a preliminary test we will simulate a lens with and without substructure and plot the residual to see if any change in the image possitions has occured. ###

### 1.1) Generate Sample Paramaters ###

In [ ]:
f = open(module_path+'/../Data-Tables/substructure_parameters_catalog.csv', 'w+')
f.close()

#Define how many lenses will be generated in the sample
sample_num = 1

#Define how many parameters each lens has
param_num = 10

param_names = ['z_lens', 'gamma_md', 'theta_E_md', 'e1_md', 'e2_md', 'center_x_md', 'center_y_md', 'gamma1_md', 'gamma2_md', 'p_center_x', 'p_center_y', 'z_source', 'mag_app_source', 'R_sersic_source',
               'n_sersic_source', 'e1_source', 'e2_source', 'center_x_source', 'center_y_source', 'z_lens_light', 'mag_app_light', 'R_sersic_light', 'n_sersic_light', 'e1_light', 'e2_light', 
               'z_point_source', 'x_point_source', 'y_point_source', 'mag_app_point_source']
#Generate the parameters to be used in the sample
param_dict = lens_parameters.perturberparameters(sample_num,module_path)
print(param_dict)

with open(module_path+'/../Data-Tables/substructure_parameters_catalog.csv', 'a') as f:
    np.savetxt(f, param_names, fmt='%s', newline=',')
    f.write('\n')
    np.savetxt(f, param_dict, fmt='%1.15f', delimiter=',')

### 1.2) Paltas Model Without Subscructure ### 
<br>
First we will simiulate a lens without substructure.

In [ ]:
WoS = ConfigHandler('../Configs/substructure_WoS_config.py')
im_WoS,metadata = WoS.draw_image()
#plt.axis('off')
plt.imshow(im_WoS,norm=simple_norm(im_WoS,stretch='log',min_cut=1e-6))
plt.savefig(module_path+'/../Images/Sub_WoS.png',bbox_inches='tight')

### 1.3) Paltas Model With Substructure ###
<br>
Next we will add substructure into the halo of the lens.

In [ ]:
WS = ConfigHandler('../Configs/substructure_WS_config.py')
im_WS,metadata = WS.draw_image()
#plt.axis('off')
plt.imshow(im_WS,norm=simple_norm(im_WS,stretch='log',min_cut=1e-6))
plt.savefig(module_path+'/../Images/Sub_WS.png',bbox_inches='tight')

### 1.4) Residual ###

In [ ]:
im_ris = im_WoS-im_WS
resid_norm = mcolors.TwoSlopeNorm(vmin=-0.025,vcenter=0,vmax=0.025)
fig, ax = plt.subplots(layout='constrained')
im2 = ax.matshow(im_ris,cmap='bwr',norm=resid_norm)
plt.imshow(im_ris, cmap='bwr',norm=resid_norm)
fig.colorbar(im2,ax=ax)
#plt.imshow(im_ris, norm=simple_norm(im_ris,stretch='log',min_cut=1e-6))
plt.savefig(module_path+'/../Images/Sub_ris.png',bbox_inches='tight')

### 2) Next we will simulate a test set of 100 lenses with substructure. ###

### 2.1) Generate Sample Parameters ###

In [ ]:
f = open(module_path+'/../Data-Tables/substructure_parameters_catalog.csv', 'w+')
f.close()

#Define how many lenses will be generated in the sample
sample_num = 101

#Define how many parameters each lens has
param_num = 10

param_names = ['z_lens', 'gamma_md', 'theta_E_md', 'e1_md', 'e2_md', 'center_x_md', 'center_y_md', 'gamma1_md', 'gamma2_md', 'p_center_x', 'p_center_y', 'z_source', 'mag_app_source', 'R_sersic_source',
               'n_sersic_source', 'e1_source', 'e2_source', 'center_x_source', 'center_y_source', 'z_lens_light', 'mag_app_light', 'R_sersic_light', 'n_sersic_light', 'e1_light', 'e2_light', 
               'z_point_source', 'x_point_source', 'y_point_source', 'mag_app_point_source']
#Generate the parameters to be used in the sample
param_dict = lens_parameters.perturberparameters(sample_num,module_path)
#print(param_dict)

with open(module_path+'/../Data-Tables/substructure_parameters_catalog.csv', 'a') as f:
    np.savetxt(f, param_names, fmt='%s', newline=',')
    f.write('\n')
    np.savetxt(f, param_dict, fmt='%1.15f', delimiter=',')

### 2.2) Paltas Model ###

In [ ]:
path = module_path+'/'

In [ ]:
im, metadata = paltas_model.PaltasModelWoS(path)
plt.axis('off')
plt.imshow(im,norm=simple_norm(im,stretch='log',min_cut=1e-6))